<a href="https://colab.research.google.com/github/pelinbalci/LLM_Notebooks/blob/main/Embedding_search_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Article&Query Embeddings and Search for Similarity

In [1]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 10.4 MB/s eta 0:00:00


In [2]:
!pip install sentence_transformers
!pip install editdistance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=380ceb3e630b48d8d89fb2835094b77ea846221501535f1e8c906779307eec73
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [3]:
import re
import numpy as np
import pandas as pd
import editdistance
from sentence_transformers import SentenceTransformer
import torch

In [4]:
device_count = torch.cuda.device_count()
if device_count > 0:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


In [32]:
# Calculate the cosine similarity for two vectors
def find_cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similarity between u and v
    Arguments:
        u -- a word vector of shape (n,)
        v -- a word vector of shape (n,)
    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    dot = np.dot(u, v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    # Avoid division by zero error
    if np.isclose(norm_u * norm_v, 0, atol=1e-32):
        return 0
    # Compute the cosine similarity
    cosine_similarity = dot / (norm_u * norm_v)

    return cosine_similarity


# Calculate the lexical similarity of the given two texts/sentences
def find_lexical_similarity(text1, text2):
    """editdistance:  the minimum number of operations required to transfer one text into the other"""
    lexical_sim = 1 - (editdistance.eval(text1, text2) / max(len(text1), len(text2)))
    return lexical_sim


def prepare_text(text):
    # lower
    text = text.lower()
    # remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    return text


def chunk_article(article):
  sentences = ["Gender equality is the goal, while gender neutrality and gender equity are practices and ways of thinking that help in achieving the goal.",
               "In 1994, the twenty-year Cairo Programme of Action was adopted at the International Conference on Population and Development (ICPD) in Cairo.",
               "The United Nations Security Council Resolution 1325 (UNSRC 1325), which was adopted on 31 October 2000, deals with the rights and protection of women and girls during and after armed conflicts.",
               "The Council of Europe's Convention on preventing and combating violence against women and domestic violence, the first legally binding instrument in Europe in the field of violence against women,[14] came into force in 2014.",
               "Gender equality is a fundamental human right.",
               "The United Nations has been actively promoting gender equality and women's empowerment.",
               "Numerous studies conducted by the United Nations have highlighted the persistent challenges of gender discrimination worldwide.",
               "One of the key areas of concern is the gender gap in access to education.",
               "Efforts are being made to bridge this gap and ensure equal educational opportunities for all genders.",
               "In the field of technology, there has been a recognized gender disparity with fewer women pursuing careers in tech-related jobs.",
               "Studies have investigated the reasons behind this gender gap, including societal stereotypes, lack of female role models, and biases in recruitment processes.",
               "Gender equality is closely tied to the United Nations Sustainable Development Goals (SDGs), particularly Goal 5.",
               "The United Nations has set specific targets and indicators to track progress toward gender equality.",
               "Studies have shown that gender discrimination exists in various forms.",
               "Efforts to promote gender equality also extend to the workplace.",
               "Education plays a vital role in empowering women and girls.",
               "Studies have demonstrated that when girls have access to quality education, they are more likely to lead healthier lives, make informed choices, and participate actively in their communities.",
               "The United Nations has emphasized the importance of gender-responsive education systems.",
               "Gender equality is not only about addressing disparities faced by women and girls but also about dismantling harmful norms and attitudes that perpetuate discrimination.",
               "Achieving gender equality also requires addressing the intersectionality of discrimination.",
               "Studies have highlighted the need for targeted policies and programs to support marginalized groups.",
               "The United Nations Women's Empowerment Principles (WEPs) provide a framework for businesses to promote gender equality in the workplace, marketplace, and community.",
               "In recent years, there has been increased attention to the role of women in peace and security efforts.",
               "Efforts to achieve gender equality require a multi-pronged approach.",
               "One of the key challenges in achieving gender equality is combating violence against women.",
               "The United Nations has called for the elimination of all forms of violence against women and girls.",
               "Gender-responsive budgeting is an important tool in advancing gender equality.",
               "Studies have shown that investing in gender equality has a positive impact on economic growth and development.",
               "The United Nations has set ambitious targets for achieving gender equality by 2030.",
               "Gender equality is not just a women's issue; it is a human rights issue that benefits society as a whole.",
               "Promoting gender equality requires changing cultural norms and stereotypes.",
               "Studies have shown that when women are economically empowered, they invest more in their families' health, education, and well-being.",
               "Efforts to achieve gender equality must also address the digital gender divide.",
               "Women in leadership roles are essential for promoting gender equality.",
               "Gender equality is closely linked to environmental sustainability.",
               "Studies have shown that when women are involved in natural resource management and environmental decision-making, conservation efforts tend to be more successful.",
               "Addressing gender-based discrimination in healthcare is crucial for achieving gender equality.",
               "Efforts to promote gender equality must also consider the unique challenges faced by LGBTQ+ individuals.",
               "Gender-sensitive reporting and data collection are essential for tracking progress toward gender equality.",
               "The United Nations has established mechanisms for monitoring and reporting on progress toward gender equality.",
               "Gender equality is not just a goal in itself but a means to achieve a more just and equitable world.",
               "Promoting gender equality requires engaging men and boys as allies in the fight against gender discrimination.",
               "Intersectional feminism acknowledges that women's experiences of discrimination vary based on their intersecting identities.",
               "In many parts of the world, child marriage remains a significant barrier to gender equality.",
               "Efforts to promote gender equality must also consider the needs of women with disabilities.",
               "Gender stereotypes in the media play a role in perpetuating gender discrimination.",
               "The United Nations promotes gender mainstreaming, which involves integrating a gender perspective into all policies, programs, and activities."]
  return sentences


def get_embeddings(sentences):
  model = SentenceTransformer(embedding_model_name)
  list_sentence_embeddings =[]
  for sen in sentences:
    sem = (prepare_text(sen))
    sentence_embeddings = model.encode(sen)
    list_sentence_embeddings.append(sentence_embeddings)
  df = pd.DataFrame({"sentences": sentences, "embedding": list_sentence_embeddings})
  return df


def similarity(query:str, df:pd.DataFrame, top_n: int=2) -> tuple[list[str], list[float]]:
    # prepare query embeddings
    model = SentenceTransformer(embedding_model_name)
    query = (prepare_text(query))
    query_embeddings = model.encode(query)

    # apply cosine similarity to two vectors
    strings_and_relatednesses = [(row["sentences"], find_cosine_similarity(query_embeddings, row["embedding"])) for i, row in df.iterrows()]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)

    # transformers_similarity_score = find_cosine_similarity(sentence_embeddings[0], sentence_embeddings[1])

    # # find lexical similarity with edit distance
    # lexical_similarity = find_lexical_similarity(text1, text2)

    # # weighted average
    # similarity_score = transformers_similarity_score * 0.6 + lexical_similarity * 0.4
    return strings[:top_n], relatednesses[:top_n]


In [33]:
# find sentence embeddings
embedding_model_name = "bert-base-uncased"
article = "XX"
sentences = chunk_article(article)
df = get_embeddings(sentences)

In [34]:
df

,sentences,embedding
0,"Gender equality is the goal, while gender neut...","[-0.09851929, 0.33918843, 0.058249637, -0.0994..."
1,"In 1994, the twenty-year Cairo Programme of Ac...","[-0.5389381, 0.09757619, -0.24887957, -0.32296..."
2,The United Nations Security Council Resolution...,"[0.1681671, -0.14870915, -0.32167298, -0.25005..."
3,The Council of Europe's Convention on preventi...,"[-0.14700334, -0.055755515, 0.02170571, -0.282..."
4,Gender equality is a fundamental human right.,"[0.35963002, 0.0007512748, -0.05180221, -0.058..."
5,The United Nations has been actively promoting...,"[-0.0050870953, -0.05482811, -0.3305806, -0.24..."
6,Numerous studies conducted by the United Natio...,"[-0.05886219, 0.21144235, -0.48438302, -0.2876..."
7,One of the key areas of concern is the gender ...,"[-0.37409475, -0.049049154, -0.25100315, -0.04..."
8,Efforts are being made to bridge this gap and ...,"[0.11336209, 0.08417587, 0.002028507, -0.08268..."
9,"In the field of technology, there has been a r...","[-0.0635689, 0.11198109, -0.11181225, 0.256939..."


# How the Search is Working?

In [39]:
model = SentenceTransformer(embedding_model_name)
query = (prepare_text("what is gender equality"))
print(query)
query_embeddings = model.encode(query)
print(query_embeddings.shape)

what is gender equality
(768,)


In [40]:
find_cosine_similarity(query_embeddings, df.iloc[0]["embedding"])

0.59695935

In [41]:
query_embeddings.shape

(768,)

In [42]:
df.iloc[0]["embedding"].shape

(768,)

In [44]:
strings, relatednesses = similarity(query, df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

relatedness=0.757


'Gender equality is a fundamental human right.'

relatedness=0.718


'Promoting gender equality requires changing cultural norms and stereotypes.'

relatedness=0.693


'Gender equality is closely linked to environmental sustainability.'

relatedness=0.680


'Efforts to achieve gender equality must also address the digital gender divide.'

relatedness=0.677


'Achieving gender equality also requires addressing the intersectionality of discrimination.'

In [60]:
strings, relatednesses = similarity(query, df, top_n=5)
print(list(strings))

['Promoting gender equality requires engaging men and boys as allies in the fight against gender discrimination.', "The Council of Europe's Convention on preventing and combating violence against women and domestic violence, the first legally binding instrument in Europe in the field of violence against women,[14] came into force in 2014.", 'Efforts are being made to bridge this gap and ensure equal educational opportunities for all genders.', 'Gender equality is the goal, while gender neutrality and gender equity are practices and ways of thinking that help in achieving the goal.', "The United Nations has been actively promoting gender equality and women's empowerment."]


# Create Prompt for GPT3.5

In [45]:
# query
query = "what is gender equality"

strings, relatednesses = similarity(query, df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    print(string)


# Prepare prompt
strings, relatednesses = similarity(query, df)

# message to model
introduction = 'Use the below articles on the gender equality to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."'

# user's question
question = f"\n\nQuestion: {query}"


message = introduction

for string in strings:
    next_article = f'\n\nWikipedia article section:\n"""\n{string}\n"""'
    if len((message + next_article + question).split()) > 300:
        break
    else:
        message += next_article

final_query = message + question


relatedness=0.757
Gender equality is a fundamental human right.
relatedness=0.718
Promoting gender equality requires changing cultural norms and stereotypes.
relatedness=0.693
Gender equality is closely linked to environmental sustainability.
relatedness=0.680
Efforts to achieve gender equality must also address the digital gender divide.
relatedness=0.677
Achieving gender equality also requires addressing the intersectionality of discrimination.


In [46]:
final_query

'Use the below articles on the gender equality to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."\n\nWikipedia article section:\n"""\nGender equality is a fundamental human right.\n"""\n\nWikipedia article section:\n"""\nPromoting gender equality requires changing cultural norms and stereotypes.\n"""\n\nQuestion: what is gender equality'

# ASK

## GPT2 Answers

In [53]:
def get_finalquery(query):

  strings, relatednesses = similarity(query, df, top_n=5)

  # message to model
  introduction = 'Use the below articles on the gender equality to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."'

  # user's question
  question = f"\n\nQuestion: {query}"


  message = introduction

  for string in strings:
      next_article = f'\n\nWikipedia article section:\n"""\n{string}\n"""'
      if len((message + next_article + question).split()) > 300:
          break
      else:
          message += next_article

  final_query = message + question
  return final_query


def get_answer(final_query, top_k=10, top_p=0.9, temperature=1, max_length=500):
  model = "gpt2"

  tokenizer = AutoTokenizer.from_pretrained(model)

  pipeline = transformers.pipeline(
      "text-generation",
      model=model,
      torch_dtype=torch.float16,
      device_map=device,
  )

  # final_query = "I liked 'Breaking Bad' and 'Band of Brothers'. Do you have any recommendations of other shows I might like?"
  sequences = pipeline(
      final_query,
      do_sample=True,
      top_k=10,
      top_p=0.9,
      temperature=1,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_length=500,  # can increase the length of sequence
  )

  return sequences[0]['generated_text'][len(final_query):]


In [56]:
query = "Wht are the studies in United Nations for gender equality?"
final_query = get_finalquery(query)
print("Final Query:")
print(final_query)
print(" ")
print("-------------------------------------------")
print("")
answer = get_answer(final_query, top_k=5, top_p=0.9, temperature=0.5, max_length=1024)
print("Answer:")
print(answer)

Final Query:
Use the below articles on the gender equality to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."

Wikipedia article section:
"""
Promoting gender equality requires engaging men and boys as allies in the fight against gender discrimination.
"""

Wikipedia article section:
"""
The Council of Europe's Convention on preventing and combating violence against women and domestic violence, the first legally binding instrument in Europe in the field of violence against women,[14] came into force in 2014.
"""

Wikipedia article section:
"""
Efforts are being made to bridge this gap and ensure equal educational opportunities for all genders.
"""

Wikipedia article section:
"""
Gender equality is the goal, while gender neutrality and gender equity are practices and ways of thinking that help in achieving the goal.
"""

Wikipedia article section:
"""
The United Nations has been actively promoting gender equality and wom

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer:


"The United Nations has been actively promoting gender equality and women's empowerment.

"The United Nations has been actively promoting gender equality and women's empowerment.

"

Question: Do the studies in United Nations for gender equality?

"The United Nations has been actively promoting gender equality and women's empowerment.

"

Question: Do the studies in United Nations for gender equality?

"The United Nations has been actively promoting gender equality and women's empowerment.

"

Question: Do the studies in United Nations for gender equality?

"The United Nations has been actively promoting gender equality and women's empowerment.

"

Question: Do the studies in United Nations for gender equality?

"The United Nations has been actively promoting gender equality and women's empowerment.

"

Question: Do the studies in United Nations for gender equality?

"The United Nations has been actively promoting gender equality and women's empowerment.

"

Question: Do the 

## T5 Answers

In [84]:
strings

('Promoting gender equality requires engaging men and boys as allies in the fight against gender discrimination.',
 "The Council of Europe's Convention on preventing and combating violence against women and domestic violence, the first legally binding instrument in Europe in the field of violence against women,[14] came into force in 2014.",
 'Efforts are being made to bridge this gap and ensure equal educational opportunities for all genders.',
 'Gender equality is the goal, while gender neutrality and gender equity are practices and ways of thinking that help in achieving the goal.',
 "The United Nations has been actively promoting gender equality and women's empowerment.")

In [68]:
final_query

'Use the below articles on the gender equality to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."\n\nWikipedia article section:\n"""\nPromoting gender equality requires engaging men and boys as allies in the fight against gender discrimination.\n"""\n\nWikipedia article section:\n"""\nThe Council of Europe\'s Convention on preventing and combating violence against women and domestic violence, the first legally binding instrument in Europe in the field of violence against women,[14] came into force in 2014.\n"""\n\nWikipedia article section:\n"""\nEfforts are being made to bridge this gap and ensure equal educational opportunities for all genders.\n"""\n\nWikipedia article section:\n"""\nGender equality is the goal, while gender neutrality and gender equity are practices and ways of thinking that help in achieving the goal.\n"""\n\nWikipedia article section:\n"""\nThe United Nations has been actively promoting gender equ

In [75]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load pre-trained T5 model and tokenizer
model_name = "t5-base"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name, model_max_length=512)  # Set model_max_length explicitly

# Splitting the context and the question with a newline character
prompt = (
    "Use the below articles on the gender equality to answer the subsequent question: "
    "Promoting gender equality requires engaging men and boys as allies in the fight against gender discrimination."
    "The Council of Europe's Convention on preventing and combating violence against women and domestic violence, "
    "the first legally binding instrument in Europe in the field of violence against women,[14] came into force in 2014. "
    "Efforts are being made to bridge this gap and ensure equal educational opportunities for all genders. "
    "Gender equality is the goal, while gender neutrality and gender equity are practices and ways of thinking that help in achieving the goal.\n"
    "What are the studies in United Nations for gender equality?"
)

# Encode the prompt to tensor
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids

# Generate a response
output = model.generate(
    input_ids,
    do_sample=True,
    top_k=10,
    top_p=0.9,
    temperature=1,
    max_length=100,
)

# Decode the generated text back to string
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)


False distances and falsehoods are false


In [100]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load pre-trained T5 model and tokenizer
model_name = "t5-base"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name, model_max_length=512)

# Define title, context and question
title = "Gender Equality Efforts"
context = (
    "Gender equality is a vital societal goal that encompasses a wide array of studies and research efforts.",
    "These studies investigate various aspects of gender disparities, including the gender wage gap, representation in leadership roles, violence against women, healthcare access, and the impact of societal norms and media representations on gender roles.",
    "Researchers employ diverse methodologies, from quantitative data analysis to qualitative sociocultural examinations, to shed light on these complex issues.",
    "The findings from these studies serve as a crucial foundation for advocacy, policy changes, and social awareness campaigns, ultimately striving to create a more equitable and inclusive world where all individuals, regardless of their gender, can enjoy equal rights and opportunities."
)
question = "What are the studies for gender equality?"

# Format the prompt using the chosen format
prompt = f"Please answer a question about the following article about {title}:\n\n{context}\n\n{question}"

# Encode the prompt to tensor
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids

# Generate a response using the specified parameters
output = model.generate(
    input_ids,
    do_sample=True,
    top_k=10,
    top_p=0.9,
    temperature=1,
    max_length=1000,
)

# Decode the generated text back to string
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)


', gender inequality, 'the gender wage gap, representation in leadership roles, violence against women, healthcare access, and the impact of societal norms and media representations on gender roles.', 'These studies serve as a crucial foundation for advocacy, policy changes, and social awareness campaigns, ultimately striving to create a more equitable and inclusive world where all individuals, regardless of their gender, can enjoy equal rights and opportunities.', 'What are the studies for gender equality?


## Bert Answers

In [76]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Load pre-trained BERT model and tokenizer
model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Define context and question
context = (
    "Use the below articles on the gender equality to answer the subsequent question: "
    "Promoting gender equality requires engaging men and boys as allies in the fight against gender discrimination."
    "The Council of Europe's Convention on preventing and combating violence against women and domestic violence, "
    "the first legally binding instrument in Europe in the field of violence against women,[14] came into force in 2014. "
    "Efforts are being made to bridge this gap and ensure equal educational opportunities for all genders. "
    "Gender equality is the goal, while gender neutrality and gender equity are practices and ways of thinking that help in achieving the goal."
)
question = "What are the studies in United Nations for gender equality?"

# Tokenize input
inputs = tokenizer(question, context, return_tensors='pt', max_length=512, truncation=True)

# Get answer
answer = model(**inputs)

# Get the most likely answer span
answer_start = torch.argmax(answer.start_logits)
answer_end = torch.argmax(answer.end_logits)

# Decode the answer
decoded_answer = tokenizer.decode(inputs['input_ids'][0][answer_start:answer_end + 1])

print(decoded_answer)


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


gender neutrality and gender equity


In [101]:
query = "Wht are the studies in United Nations for gender equality?"
strings, relatednesses = similarity(query, df, top_n=5)
context = ''
for item in strings:
    context = context + item

print(context)

from transformers import pipeline
question_answerer = pipeline("question-answering", model='distilbert-base-uncased-distilled-squad')

result = question_answerer(question=query, context=context)
print(
f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Promoting gender equality requires engaging men and boys as allies in the fight against gender discrimination.The Council of Europe's Convention on preventing and combating violence against women and domestic violence, the first legally binding instrument in Europe in the field of violence against women,[14] came into force in 2014.Efforts are being made to bridge this gap and ensure equal educational opportunities for all genders.Gender equality is the goal, while gender neutrality and gender equity are practices and ways of thinking that help in achieving the goal.The United Nations has been actively promoting gender equality and women's empowerment.
Answer: 'The United Nations has been actively promoting gender equality and women's empowerment.', score: 0.0922, start: 573, end: 660
